# Lab 1: Experiment Tracking
## Exercise 3: Add configuration and save model

Now that our experiment is tracked, we can add more information to it.

Connect a set of [hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)) (configuration) to your experiment using [Task.connect](https://clear.ml/docs/latest/docs/references/sdk/task/#connect). See docs here: https://clear.ml/docs/latest/docs/fundamentals/hyperparameters#explicit-logging. You should see them appear in ClearML under CONFIGURATION > HYPERPARAMETERS > General.

Save the tensorflow model after training. ClearML will pick it up automatically if you set `Task.init(output_uri=True)`. See docs here: https://www.tensorflow.org/guide/keras/serialization_and_saving. You should see the model appear in ClearML under ARTIFACTS > OUTPUT MODELS.

In [1]:
import os
from dotenv import load_dotenv

#%pip install -q clearml nbconvert

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

load_dotenv()

CLEARML_API_ACCESS_KEY=os.getenv("CLEARML_API_ACCESS_KEY")
CLEARML_API_SECRET_KEY=os.getenv("CLEARML_API_SECRET_KEY")

if CLEARML_API_ACCESS_KEY is None:
    raise KeyError("CLEARML_API_ACCESS_KEY")

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml


In [2]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from clearml import Task, Logger

task = Task.init(project_name='vives-mlops-workshop', task_name='exercise3', output_uri=True)

# hyperparameters
config = {
    "conv1": 4,
    "conv2": 8,
    "conv3": 8,
    "dense1": 16,
}
# TODO connect config using Task.connect()
prev_task = Task.get_task(task_id='5df484389a6a40b7b6cdbb12be4e7f38')
task.connect(config)
# ---

# download the dataset
(images, labels), _ = keras.datasets.cifar10.load_data()

# there are 10 classes of images
all_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# choose four classes (feel free to change this!)
class_names = ["bird", "cat", "deer", "dog"]
print("Class names:", class_names)

# only keep images of these classes
class_indexes = [all_classes.index(c) for c in class_names]
to_keep = np.array([l in class_indexes for l in labels])
images = images[to_keep]
labels = labels[to_keep]

# change indexes from 10 to 2 classes
labels = np.array([class_indexes.index(l) for l in labels])

# normalize pixels between 0 and 1
images = images / 255.0

# split into train and test set
split = round(len(images) * 0.8)
train_images = images[:split]
train_labels = labels[:split]
test_images = images[split:]
test_labels = labels[split:]
print("Number of train images:", len(train_images))
print("Number of test images:", len(test_images))

# ---

# create neural network
model = keras.models.Sequential()
model.add(keras.Input(shape=(32, 32, 3)))

# convolutional layers
model.add(layers.Conv2D(config["conv1"], (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(config["conv2"], (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(config["conv3"], (3, 3), activation="relu"))

# add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(config["dense1"], activation="relu"))
model.add(layers.Dense(4, activation="softmax"))

# ---

# compile and train the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

# ---

# report accuracy to ClearML
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
logger = Logger.current_logger()

for i in range(len(accuracy)):
    logger.report_scalar("training", "accuracy", accuracy[i], iteration=i)
    logger.report_scalar("training", "val_accuracy", val_accuracy[i], iteration=i)

# TODO save model to disk
model.save('./exercise3.keras')

# close ClearML task
task.close()

ClearML Task: created new task id=30f9fe02bf9343d9a36d04f792e00046
2024-06-03 13:52:56,513 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/156920b4140a41ed894eba6f84846cb0/experiments/30f9fe02bf9343d9a36d04f792e00046/output/log
Class names: ['bird', 'cat', 'deer', 'dog']
Number of train images: 16000
Number of test images: 4000
Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3132 - loss: 1.3469 - val_accuracy: 0.4198 - val_loss: 1.2268
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4366 - loss: 1.2095 - val_accuracy: 0.4615 - val_loss: 1.1685
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4841 - loss: 1.1482 - val_accuracy: 0.4837 - val_loss: 1.1445
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4954 - loss: 1.1264 - val_accuracy: 0.5035 - val_loss: 1.1133
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5252 - loss: 1.0928 - 